# LittleLight 4

## 前情提要

在我的感覺裡，這一週過得很快，有很高興的事情，但也有點痛苦。雖然每天都有一點小小的進展，但突然間有一種很久很久沒有休息、放假了的感覺。
除了在行政事務上繼續磨耗自己之外，這一週我花了很多時間在標記相關的工作上。本著一個為人為己開方便的小心願，我每天都在寫 code ，希望真的可以幫上點忙。因此，我這週先跳出Ｒ，分享一下我用 Python 做的一個小小的互動(介面？程式？)，試著解決我在工作任務上遇到的一些問題。

## 本週主題說明

* 在標記任務中遇到的狀況 

        在判別雙字詞和其組成字之間的關係時，除了直覺之外，也需要回頭看 CWN 裡面的資料來幫助判斷。雖然 Sean 大大已經完成了我們會使用的 code 教學與示範，但是因為截稿時間在即，且需判斷的字詞不少，那麼 csv 檔每列都會變成很胖很不好判斷。如果沒有 web 介面可呈現同一個 lemma 下的 senses ，我可以想見那會需要很多很多次的手動 run code。    


* 主要提問 

        有什麼辦法可以來簡化這樣的流程，便利標記者查找詞意與例句呢？
     
     
* 解決方法

        運用使用者自行輸入的功能來做一個小幫手（可循環執行的程式），簡化查找的流程。
        
* 預期結果  

        有了小幫手之後，查找 CWN 內的詞義與例句的流程將會變為：  
            run 小幫手的 code >> 依照指示選擇功能 >> 輸入欲查詢的詞彙 >> 取得結果 >> 繼續查找直到輸入跳出暗號  
        

## CwnGraph 小幫手的誕生紀實

現在就一起來看我是怎麼一層一層堆疊出親愛的 CwnGraph 小幫手的吧！  

源自於遙遠的一個記憶，記得那次 Richard 帶大家讀書會練習了使用 Python 的某些功能寫一個倒數計時器。我對於它的印象是：好好玩啊！它可以自己就這樣一直玩下去耶！！！  我記得那次課堂的作業我異想天開的寫了一個親屬關係判定的程式，為了完成這個程式花了很多時間去找親屬稱謂、親等以及設計問答串接，很花時間，但是我覺得好好玩，寫完後就真的可以幫忙算親等耶！     

因此，當碰上缺乏 web 介面工具程式能力不足且時間有限的狀況時，我就想到了這個小方法： 使用 `input()` 並搭配迴圈執行來製作一個搜尋小幫手！  

接下來，我們就直接看 code 囉！  


In [6]:
# 這邊是一些必須執行的程式（from Sean） 
import pickle
from CwnGraph import CwnBase, CwnAnnotator
from CwnGraph import CwnRelationType

In [7]:
# 本機上安裝好必備資源後，建一個 Base 來搜尋
cwn = CwnBase()

In [8]:
# 這邊就是 CwnGraph 小幫手的骨架囉～ （這是進化到第三版的 code ）

from pprint import pprint

print("CwnGraph 資料查詢小幫手")
print("="*60)

switch = True
while switch == True:
    print("::: 小幫手功能說明 :::","--查詢詞義 >>>>>> 請輸入 a ","--查詢詞義與例句 >>>>>> 請輸入 b ", "--結束小幫手 >>>>>> 請輸入 1 ", sep="\n", end="\n\n\n")
    item = input('<功能選擇> 請輸入英文字母：')
    if item == "a":
        findsenses = True
        while findsenses == True:
            lookingsense = input('[a] 查詢詞義 >> 請輸入欲查詢的詞(跳出請輸入 2 )：')
            if lookingsense not in ["1", "2", "b"]:
                center = "^"+lookingsense+"$"
                lemmas = cwn.find_lemma(center)
                senses = cwn.find_senses(lemma=center)
                print("="*20,'\n','[[[查詢結果]]]','\n')
                print("所查詞彙:",lookingsense, "\n", "詞彙(lemma)數量:", len(lemmas), "\n", "詞義(sense)數量:",len(senses), "\n")
                for lemm in lemmas:
                    lemsense = lemm.senses
                    print(lemm)
                    if len(lemsense) > 0:
                        for i in range(len(lemsense)):
                            pos = lemsense[i].data()["pos"]
                        #examples = lemsense[i].examples[0]
                            print(" ", "[詞性]", pos, " ", lemsense[i])
                print("="*30)
                    
            elif lookingsense == "2":
                print("\n",'跳出詞義查詢～～～',end="\n\n")
                print("="*30)
                findsenses = False
            
            elif lookingsense == "1":
                print("\n", '再會～～～小幫手退場囉～～～',end="\n\n")
                findsenses = False
                switch = False
                
            elif lookingsense == "b":
                print("\n", '跳至查詢詞義與例句～～～',end="\n\n")
                print("="*30)
                findsenses = False 
                item = "b"
                
    if item == "b":    
        findall = True
        while findall == True:
            name = input('[b] 查詢詞義與例句 >> 請輸入欲查詢的詞(跳出請輸入 2 )：')
            seed = "^"+name+"$"
            lemman= len(cwn.find_senses(lemma=seed))
            result = {x: x.all_examples() for x in cwn.find_senses(seed)}
            
            if name not in ["1", "2", "a"]:
                print("="*20,'\n','[[[查詢結果]]]',"\n\n", "「", name, "」的詞義數量：",lemman,"\n------")
                for k, v in result.items():
                    spos = k.data()["pos"]
                    print("詞性:", spos,"  ", k,"\n", "::例句::", "\n", v,"\n","------","\n")
                
                print("="*30)

            elif name == "2":
                print("\n", "跳出詞義與例句查詢～～～",end="\n\n")
                print("="*30)
                findall = False
                
            elif name == "1":
                print("\n", '再會～～～小幫手退場囉～～～',end="\n\n")
                findall = False
                switch = False
                
            elif name == "a":
                print("\n", '跳至查詢詞義～～～',end="\n\n")
                print("="*30)
                findall = False
                item = "a" #不知如何跳回去 [a] 查詢詞義＠＠
                findsenses = True
                
               
    if item == "1":
        print("\n", '再會～～～小幫手退場囉～～～')
        switch = False
    
    

CwnGraph 資料查詢小幫手
::: 小幫手功能說明 :::
--查詢詞義 >>>>>> 請輸入 a 
--查詢詞義與例句 >>>>>> 請輸入 b 
--結束小幫手 >>>>>> 請輸入 1 


<功能選擇> 請輸入英文字母：a
[a] 查詢詞義 >> 請輸入欲查詢的詞(跳出請輸入 2 )：梅花
 [[[查詢結果]]] 

所查詞彙: 梅花 
 詞彙(lemma)數量: 1 
 詞義(sense)數量: 1 

<CwnLemma: 梅花_1>
  [詞性] Na   <CwnSense[04065701](梅花): 梅樹或梅樹的花。>
[a] 查詢詞義 >> 請輸入欲查詢的詞(跳出請輸入 2 )：b

 跳至查詢詞義與例句～～～

[b] 查詢詞義與例句 >> 請輸入欲查詢的詞(跳出請輸入 2 )：梅花
 [[[查詢結果]]] 

 「 梅花 」的詞義數量： 1 
------
詞性: Na    <CwnSense[04065701](梅花): 梅樹或梅樹的花。> 
 ::例句:: 
 ['漸江臨終前，告訴人們在前多植<梅花>，誓死不向命運低頭。', '<梅花>天寒地凍中，北風正凜冽；秋草已枯萎，百花盡凋謝。', '春不見春，芒鞋踏破嶺頭雲，歸來偶過<梅花>下，春在枝頭已十分。', '你會喜歡松柏，喜歡竹子，喜歡<梅花>和蘭草。自然界的一切，在你的心中，都變得很美，很可愛。', '在冰天雪地的冬天，沒有一點兒生氣，只有<梅花>仍然在風雪中盛開。', '玄宗瞧著深愛<梅花>的江妃，就像<梅花>令人楚楚可憐的那種淒美動人。', '梅石，當初是由於梅石沃口產像<梅花>般的石子而得名，這是一個已沒落的小村落。'] 
 ------ 

[b] 查詢詞義與例句 >> 請輸入欲查詢的詞(跳出請輸入 2 )：1

 再會～～～小幫手退場囉～～～



## 再回頭看一看做了什麼  

  
[週一]   
    下午開完會就開始寫，吃過飯後大概花了兩三個小時才做完初版的設計。小幫手的功能包含了 [a]詞義查詢 和 [b]詞義與例句查詢 ，不過呈現的畫面是很粗糙的，還在摸索如何做更簡潔易讀的呈現。    


[週二]  
    開始使用小幫手在做雙字詞的標記作業，發現在 lemma 上應該直接鎖定輸入詞彙就好，不需要太過雜的資訊，例如輸入電腦後查詢的結果不應該包含電腦化等其他包含電腦一詞的 lemma 並將之修正等。    


[週三]  
    帶著前一天努力標記時的困惑，跟謝老闆討論後覺得詞性很重要(雖然我週二做的時候就有在看詞性了但沒力氣做)，這個資料也必須要搜尋時拿夠被抓出來，趁著下班空檔研究 code 並改寫了一下呈現方式。  

~~我的媽呀，把腦中的想法寫出來而且還可以用，超興奮！自己用還覺得很實用ＸＤＤ~~ 

## 學習筆記

1. 找了一下資料才找到可以讓程式一直執行的語法 `while switch == True` ，這次還突然會了在不同功能區持續啟動運行的寫法，覺得自己又跨出了一步！  

2. 重新溫習了 Python 流程控制的語法，但是還是不太知道如何在 b 功能的區域直接跳回 a 功能區。 ＱＱ  

3. 再次使用到資料型態查找以及熟悉 `dict` 的取值與運用，想通的那一刻覺得自己腦袋還堪用！  

4. 在調整資料呈現方式與抓 pos 的時候，是了超多次的，到後來竟覺得自己的迴圈邏輯變好了！（~~希望不要是誤解~~）  

5. 調整資料呈現方式其實好像很看人，體會到到最後細節的微調其實真的是自己在為難自己。下次要記得先求「實用功能的實現」，再求「呈現的簡潔易讀」，微調呈現方式只是個人的花招，沒有要永久存在的、可供大家自由修改的小工具盡量不要花超過兩小時在做呈現細節的微調。